<a href="https://colab.research.google.com/github/Ravindra-Nayak/Agrim_Dashboard/blob/main/n_p_k_deficiency_detection_and_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#The model will detect the nitrogen,phosphorus,potassium deficiency and will also calculate deficiency levels of each parameters and will build the clusters of the given dataset
# importing libraries for the given dataset

import pandas as pd
from sklearn.cluster import KMeans

In [ ]:
# Step 1: Load the Agrim dataset
data = pd.read_csv('/content/agridatas.csv')

<ipython-input-73-9a8507ea1f3e>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/agridatas.csv')


In [ ]:
# Step 2: Select relevant features
X = data[['nitrogen', 'phosphorus', 'potassium', 'ph', 'temperature', 'humidity']]

In [ ]:
# Step 3: Choose the number of clusters (we can determine it using the Elbow Method)
k = 3  # we can also Replace with our chosen number of clusters

In [ ]:
# Step 4: Initialize K-Means
kmeans = KMeans(n_clusters=k, random_state=42)

In [ ]:
# Step 5: Fit K-Means to the data
kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3, random_state=42)

In [ ]:
# Step 6: Add cluster labels to the original dataset
data['cluster'] = kmeans.labels_

In [ ]:
# Step 7: Define deficiency detection rules for all clusters
def detect_deficiency(cluster, nutrients):
    deficiency_messages = []
    deficiency_levels = {'nitrogen': 0, 'phosphorus': 0, 'potassium': 0}

    if nutrients['nitrogen'] < 4.0:
        deficiency_messages.append(f'Nitrogen deficiency detected. Deficiency level: {4.0 - nutrients["nitrogen"]:.2f}')
        deficiency_levels['nitrogen'] = 4.0 - nutrients['nitrogen']

    if nutrients['phosphorus'] < 2.0:
        deficiency_messages.append(f'Phosphorus deficiency detected. Deficiency level: {2.0 - nutrients["phosphorus"]:.2f}')
        deficiency_levels['phosphorus'] = 2.0 - nutrients['phosphorus']

    if nutrients['potassium'] < 1.0:
        deficiency_messages.append(f'Potassium deficiency detected. Deficiency level: {1.0 - nutrients["potassium"]:.2f}')
        deficiency_levels['potassium'] = 1.0 - nutrients['potassium']

    if not deficiency_messages:
        deficiency_messages.append('No deficiency detected.')

    return ', '.join(deficiency_messages), deficiency_levels


In [ ]:
# Step 8: Apply the deficiency detection function to the dataset
data[['deficiency_message', 'deficiency_levels']] = data.apply(
    lambda row: pd.Series(detect_deficiency(row['cluster'], {'nitrogen': row['nitrogen'], 'phosphorus': row['phosphorus'], 'potassium': row['potassium']})), axis=1)

In [ ]:
# Step 9: Summarize the results
print("Summary:")
print(data[['nitrogen', 'phosphorus', 'potassium', 'ph', 'temperature', 'humidity', 'cluster', 'deficiency_message', 'deficiency_levels']])


Summary:
        nitrogen  phosphorus  potassium    ph  temperature  humidity  cluster  \
0              4           5         11  7.29         23.7      18.5        2   
1              2           4          8  7.94         21.8      12.3        1   
2              5           8         16  7.00         28.6      14.3        2   
3              5           7         14  7.17         28.2      13.7        2   
4              3           4          9  7.00         21.8      13.1        0   
...          ...         ...        ...   ...          ...       ...      ...   
131153         5           7         16  7.77         22.2      16.4        2   
131154         5           8         16  7.33         28.5      16.1        2   
131155         2           3          7  7.30         27.0      11.0        1   
131156         3           4          9  7.22         24.6      11.8        0   
131157         2           3          7  7.47         22.6      11.0        1   

                  

In [ ]:
# Step 10: Save or deploy the model for future use

# 1st Example usage with a new dataset
new_data = pd.DataFrame({'nitrogen': [3.8], 'phosphorus': [1.5], 'potassium': [0.8], 'ph': [6.3], 'temperature': [27.0], 'humidity': [40.0]})
new_cluster = kmeans.predict(new_data)[0]
new_deficiency_message, new_deficiency_levels = detect_deficiency(new_cluster, {'nitrogen': new_data['nitrogen'][0], 'phosphorus': new_data['phosphorus'][0], 'potassium': new_data['potassium'][0]})

print("\nExample with New Dataset:")
print(f'Cluster: {new_cluster}')
print(f'Deficiency Detection Result: {new_deficiency_message}')
print(f'Deficiency Levels: {new_deficiency_levels}')


Example with New Dataset:
Cluster: 2
Deficiency Detection Result: Nitrogen deficiency detected. Deficiency level: 0.20, Phosphorus deficiency detected. Deficiency level: 0.50, Potassium deficiency detected. Deficiency level: 0.20
Deficiency Levels: {'nitrogen': 0.20000000000000018, 'phosphorus': 0.5, 'potassium': 0.19999999999999996}


In [ ]:

# model test for with 2nd Example usage with a new dataset
new_data = pd.DataFrame({'nitrogen': [8], 'phosphorus': [1.5], 'potassium': [0.8], 'ph': [6.3], 'temperature': [27.0], 'humidity': [40.0]})
new_cluster = kmeans.predict(new_data)[0]
new_deficiency_message, new_deficiency_levels = detect_deficiency(new_cluster, {'nitrogen': new_data['nitrogen'][0], 'phosphorus': new_data['phosphorus'][0], 'potassium': new_data['potassium'][0]})

print("\nExample with New Dataset:")
print(f'Cluster: {new_cluster}')
print(f'Deficiency Detection Result: {new_deficiency_message}')
print(f'Deficiency Levels: {new_deficiency_levels}')


Example with New Dataset:
Cluster: 2
Deficiency Detection Result: Phosphorus deficiency detected. Deficiency level: 0.50, Potassium deficiency detected. Deficiency level: 0.20
Deficiency Levels: {'nitrogen': 0, 'phosphorus': 0.5, 'potassium': 0.19999999999999996}


In [ ]:

# model test for with 3rd Example usage with a new dataset
new_data = pd.DataFrame({'nitrogen': [4], 'phosphorus': [2], 'potassium': [1], 'ph': [6.3], 'temperature': [27.0], 'humidity': [40.0]})
new_cluster = kmeans.predict(new_data)[0]
new_deficiency_message, new_deficiency_levels = detect_deficiency(new_cluster, {'nitrogen': new_data['nitrogen'][0], 'phosphorus': new_data['phosphorus'][0], 'potassium': new_data['potassium'][0]})

print("\nExample with New Dataset:")
print(f'Cluster: {new_cluster}')
print(f'Deficiency Detection Result: {new_deficiency_message}')
print(f'Deficiency Levels: {new_deficiency_levels}')


Example with New Dataset:
Cluster: 2
Deficiency Detection Result: No deficiency detected.
Deficiency Levels: {'nitrogen': 0, 'phosphorus': 0, 'potassium': 0}
